# [Optimizing the model parameters](https://learn.microsoft.com/en-us/training/modules/intro-machine-learning-pytorch/6-optimization)

In [17]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [18]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [21]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [23]:
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS device built: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS device available: {torch.backends.mps.is_available()}") # True 여야 합니다.
model = NeuralNetwork().to(device)

PyTorch version:2.0.0
MPS device built: True
MPS device available: True


## 1.Setting hyperparameters

- Number of epochs
> The number times the entire training dataset is pass through the network.
- Batch Size
> The number of data samples seen by the model in each eposh. Iterates are the number of batches needs to compete an epoch.
- Learning Rate
> The size of steps the model match as it searchs for best weights that will produce a higher model accuracy.    
> - Smaller values means the model will take a longer time to find the best weights    
> - Larger values may result in the model step over and misses the best weights 

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 2.Add an optimization loop

- The Train Loop
> iterate over the training dataset and try to converge to optimal parameters.   
- The Validation/test Loop
> iterate over the test dataset to check if model performance is improving.

### Add a loss function
- `nn.MSELoss` (Mean Square Error) used for regression tasks
- `nn.NLLLoss` (Negative Log Likelihood) used for classification
- `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`

In [9]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Backpropagation
- optimizer.zero_grad()
> 역전파 단계를 실행하기 전에 각 파라미터들의 변화도(gradient)를 0으로 재설정
- loss.backward()
> 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도 계산
- optimizer.step()
> 변화도를 계산한 뒤에 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

## 3.Full implementation

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [27]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            _, pred = torch.max(pred, 1)
            correct += (pred == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.149955  [    0/60000]
loss: 1.166444  [ 6400/60000]
loss: 1.048303  [12800/60000]
loss: 1.197846  [19200/60000]
loss: 0.997716  [25600/60000]
loss: 1.169582  [32000/60000]
loss: 0.994092  [38400/60000]
loss: 1.049650  [44800/60000]
loss: 1.110338  [51200/60000]
loss: 1.219340  [57600/60000]
Test Error: 
 Accuracy: 66.3%, Avg loss: 0.017298 

Epoch 2
-------------------------------
loss: 1.117510  [    0/60000]
loss: 1.139516  [ 6400/60000]
loss: 1.018969  [12800/60000]
loss: 1.171415  [19200/60000]
loss: 0.976580  [25600/60000]
loss: 1.144439  [32000/60000]
loss: 0.970844  [38400/60000]
loss: 1.031781  [44800/60000]
loss: 1.084307  [51200/60000]
loss: 1.202661  [57600/60000]
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.016959 

Epoch 3
-------------------------------
loss: 1.089572  [    0/60000]
loss: 1.116546  [ 6400/60000]
loss: 0.994298  [12800/60000]
loss: 1.148610  [19200/60000]
loss: 0.958969  [25600/60000]
loss: 1.123093  [32000/600

## 4.Saving Models

In [29]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 5. Loading Models

In [30]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

### Predictions

In [31]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
